In [2]:
from functools import reduce
import gc
import numpy as np

"""
STVM 계산(3차로 점선) + 지상부 추가 _ 변곡점 계산
    집계시간 : 1분
    분석시간 1800~6900 -> 6900까지 데이터 수집 후 5400으로 가공
    검지기 : 280개
"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 6900
collection_time = 60

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 140

# 지상부_진출 검지기
exit_line = 275
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )

    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)

    return speed_mean_df

# 밀도
def density_mean(original_df):
    copy_df = original_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):
    copy_df = original_df.copy()
    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C
    max_capacity = 6600
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        copy_df[copy_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0값으로 fix
    main_df["RFR"] = 0
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    copy_df = original_df.copy()
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = copy_df[copy_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진출)에 들어온 차량의 번호, 시간
    exit_df = copy_df[copy_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )

    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / collection_time)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
    merged_counts["1-F(outrate)"] = 1 - merged_counts["F(outrate)"]
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )

    merged_df["StartTime"] = merged_df["TimeGroup"].str.split("~").str[0].astype(int)
    merged_df = merged_df[merged_df["StartTime"] < 5400]

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["1-F(outrate)"]
    )
    display("merged_df : ", merged_df)
    merged_df = modify_frame(merged_df)
    display("merged_df2 : ", merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df = modify_df[modify_df["New_Measurement"] != 276]
    modify_df["New_Measurement"] = modify_df.groupby("TimeGroup").cumcount() + 1
    return modify_df

def calculate_avg(df):

    def avg_range(df, low, high):
        cols = [c for c in df.columns if low <= c <= high]
        return df[cols].mean(axis=1) if cols else np.nan

    avg_df = pd.DataFrame(index=df.index)
    avg_df["지상부_진입"] = avg_range(df, -5, -1)
    avg_df["진입부"] = avg_range(df, 1, 30)
    avg_df["본선부"] = avg_range(df, 31, 230)
    avg_df["진출부"] = avg_range(df, 231, 260)
    avg_df["지상부_진출"] = avg_range(df, 261, 265)

    return avg_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "1-F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\network_version2.13.0"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, collection_time)
                labels = [f"{start}~{start+collection_time}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)
                measurement_list = [
                    # 지상부_진입
                    6,
                    # 지상부-진입부
                    9, 10, 11,
                    # 진입부 - 본선부
                    39, 40, 41,
                    # 본선부
                    140,
                    # 본선부 - 진출부
                    239,240,241,
                    # 진출부 - 지상부
                    269, 270,271,
                    # 지상부_진출
                    275, 276
                ]
                original_df = original_df[original_df["New_Measurement"].isin(measurement_list)]

                # 평균속도
                speed_df = speed_mean(original_df)
                #display("speed_df : ", speed_df.head(15))

                # 밀도
                density_df = density_mean(speed_df)
                #display("density_df : ", density_df.head(15))

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)
                #display("heavy_df : ", heavy_df.head(15))

                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)
                #display("entry_saturation_df : ", entry_saturation_df.head(15))

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)
                #display("rfr_df : ", rfr_df.head(15))

                # 진출 정상성
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # 변수별 TimeGroup별 평균
                #variable_time_df = variable_timegroup_avg(stvm_df)
                #save_to_excel(variable_time_df, folder_path, "변수별 평균", i)

                # 변수별 전체 평균
                #variable_total_df = variable_total_avg(variable_time_df)
                #save_to_excel(variable_total_df, folder_path, "변수별 총 평균", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도, 밀도 전체 평균값
                #speed_density_df = speed_density_avg(density_df)
                #save_to_excel(speed_density_df, folder_path, "속도_밀도 평균값", i)

                # 속도 변화 피봇
                #delta_speed_pivot_df = pivot_table(speed_df, "delta_V", preprocess=modify_frame)
                #save_to_excel(delta_speed_pivot_df, folder_path, "속도변화량", i)

                # 밀도 변화 피봇
                #delta_density_pivot_df = pivot_table(density_df, "delta_K", preprocess=modify_frame)
                #save_to_excel(delta_density_pivot_df, folder_path, "밀도변화량", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 가중평균속도
                #weighted_speed_df = weighted_avg_speed(original_df)
                #weighted_speed_df = pivot_table(weighted_speed_df, "Weighted_Avg_Speed", preprocess=modify_frame)
                #save_to_excel(weighted_speed_df, folder_path, "가중평균속도", i)
                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

'merged_df : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,6,-0.017804,0.048072,0.014706,0.123636,0,-0.058824,1800,0.109786
1,1800~1860,9,0.001336,-0.029867,0.028571,0.123636,0,-0.058824,1800,0.064853
2,1800~1860,10,0.009765,-0.024234,0.029412,0.123636,0,-0.058824,1800,0.079755
3,1800~1860,11,-0.017931,0.063852,0.014925,0.123636,0,-0.058824,1800,0.125659
4,1800~1860,39,0.002068,0.026449,0.085714,0.123636,0,-0.058824,1800,0.179044
...,...,...,...,...,...,...,...,...,...,...
955,5340~5400,269,0.012154,-0.068144,0.045455,0.138182,0,0.394737,5340,0.522383
956,5340~5400,270,0.017234,-0.064319,0.048193,0.138182,0,0.394737,5340,0.534027
957,5340~5400,271,0.001359,-0.077203,0.050633,0.138182,0,0.394737,5340,0.507707
958,5340~5400,275,-0.001575,0.029018,0.041096,0.138182,0,0.394737,5340,0.601458


'merged_df2 : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,1,-0.017804,0.048072,0.014706,0.123636,0,-0.058824,1800,0.109786
1,1800~1860,2,0.001336,-0.029867,0.028571,0.123636,0,-0.058824,1800,0.064853
2,1800~1860,3,0.009765,-0.024234,0.029412,0.123636,0,-0.058824,1800,0.079755
3,1800~1860,4,-0.017931,0.063852,0.014925,0.123636,0,-0.058824,1800,0.125659
4,1800~1860,5,0.002068,0.026449,0.085714,0.123636,0,-0.058824,1800,0.179044
...,...,...,...,...,...,...,...,...,...,...
954,5340~5400,11,-0.040824,0.555008,0.050847,0.138182,0,0.394737,5340,1.097949
955,5340~5400,12,0.012154,-0.068144,0.045455,0.138182,0,0.394737,5340,0.522383
956,5340~5400,13,0.017234,-0.064319,0.048193,0.138182,0,0.394737,5340,0.534027
957,5340~5400,14,0.001359,-0.077203,0.050633,0.138182,0,0.394737,5340,0.507707


'z_score_df : '

...

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,6,-0.048252,0.236116,0.294118,0.123636,0,-0.382353,1800,0.223265
1,1800~1860,9,-0.001451,-0.011065,0.337500,0.123636,0,-0.382353,1800,0.066267
2,1800~1860,10,-0.005171,-0.020250,0.354430,0.123636,0,-0.382353,1800,0.070293
3,1800~1860,11,0.023587,-0.175297,0.363636,0.123636,0,-0.382353,1800,-0.046790
4,1800~1860,39,-0.002798,-0.012622,0.338462,0.123636,0,-0.382353,1800,0.064325
...,...,...,...,...,...,...,...,...,...,...
955,5340~5400,269,-0.010223,0.059613,0.243902,0.127273,0,0.242857,5340,0.663422
956,5340~5400,270,0.492604,-0.368982,0.244186,0.127273,0,0.242857,5340,0.737938
957,5340~5400,271,0.343829,-0.485531,0.283951,0.127273,0,0.242857,5340,0.512378
958,5340~5400,275,0.029369,-0.097922,0.285714,0.127273,0,0.242857,5340,0.587291


'merged_df2 : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,1,-0.048252,0.236116,0.294118,0.123636,0,-0.382353,1800,0.223265
1,1800~1860,2,-0.001451,-0.011065,0.337500,0.123636,0,-0.382353,1800,0.066267
2,1800~1860,3,-0.005171,-0.020250,0.354430,0.123636,0,-0.382353,1800,0.070293
3,1800~1860,4,0.023587,-0.175297,0.363636,0.123636,0,-0.382353,1800,-0.046790
4,1800~1860,5,-0.002798,-0.012622,0.338462,0.123636,0,-0.382353,1800,0.064325
...,...,...,...,...,...,...,...,...,...,...
954,5340~5400,11,-0.507589,1.448937,0.397059,0.127273,0,0.242857,5340,1.708536
955,5340~5400,12,-0.010223,0.059613,0.243902,0.127273,0,0.242857,5340,0.663422
956,5340~5400,13,0.492604,-0.368982,0.244186,0.127273,0,0.242857,5340,0.737938
957,5340~5400,14,0.343829,-0.485531,0.283951,0.127273,0,0.242857,5340,0.512378


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
TimeGroup,,,,,,,,,,,,,,,
1800~1860,65.732629,60.742544,60.861252,57.805091,60.685263,61.294314,70.072365,56.058619,56.892602,60.757044,63.676125,60.994643,61.849593,65.516747,57.387783
1860~1920,66.908355,69.727390,66.572308,66.345565,70.114062,70.114033,61.920622,59.172325,72.050966,72.212280,97.807227,64.198969,70.116629,68.030689,69.347106
1920~1980,67.370361,66.035427,65.168295,70.503685,67.793957,67.652433,56.602033,67.138787,67.568823,66.583456,85.806372,71.047150,70.998997,72.150309,67.676996
1980~2040,52.777464,52.792713,52.694289,52.639094,52.881522,52.736981,54.527355,52.576875,52.807065,52.867906,53.906829,52.874796,52.719747,53.119979,52.607692
2040~2100,70.716669,70.788290,71.278818,71.751789,72.097378,70.544538,83.273730,71.665928,71.919967,80.707364,53.685022,81.493469,72.375086,87.913621,70.577456
2100~2160,92.758380,89.888676,89.410666,83.502633,85.634332,88.027813,95.794775,80.353391,92.132672,93.136559,97.517794,91.963970,92.767754,90.818396,90.185834
2160~2220,71.821184,72.078732,72.418428,71.622514,71.595079,71.727864,70.218880,71.970601,70.653653,71.105289,70.881073,71.824692,70.821009,86.577324,72.013631
2220~2280,71.009911,70.825170,70.518982,66.239885,70.791876,68.360535,72.354357,71.014562,71.058068,70.471737,58.290689,70.897039,72.138969,66.132628,69.103511
2280~2340,54.011336,57.609649,55.238479,71.765017,57.945916,62.119923,52.549679,57.028612,58.527410,56.126997,98.307439,56.588468,56.724559,61.862239,57.193723
